_python notebook №2_

## Стадия обученяи сетей с применением Transfer learning

In [1]:
#pip install nbimporter

In [2]:
import nbimporter
import datasets_bilder # загрузка datasets_bilder.py из репозитория

In [5]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
import random
import seaborn as sns
import torch,torchvision
from  matplotlib import pyplot as plt
%matplotlib inline

In [6]:
# Для воспроизводимости результатов зафиксируем сиды:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [17]:
print('В репозитории находятся следующие файлы:', os.listdir())

В репозитории находятся следующие файлы: ['.git', '.gitignore', '.ipynb_checkpoints', 'chest_xray', 'datasets_bilder.py', 'models', 'Preprocessing_&_custom_DL_model.ipynb', 'README.md', 'Transfer_learning_ResNet_training.ipynb', '__pycache__']


Для реализации предобработки данных я написал отельный файл datasets_bilder.py в котором описал создание класса preprocessing_func, который в методе init генерит 3 датасета: dataset_test, dataset_train, dataset_test и сохраняет их значения в атрибутах test, train, val объекта класса. Атрибуты mean_nums и std_nums дают возможность получить величины средних значений и ско для всего train датасета. <br><br>
По сути данный код является полной копией кода из файла Preprocessing_&_custom_DL_model.ipynb, но только представленным в виде функции инициализации объекта класса. Так что для ознакомления с реализацией предобработки советую рассмотреть тот python notebook.<br> <br>
Единственным важным дополнением в предобработке, представленной в файле datasets_bilder.py, можно назвать возможность выбора размерности данных, которые мы хотим получать на выходе. Если при задании объекта класса указать значение size, то можно самому выбрать размер получаемых после трасформации Resize изображений (по умолчанию - квадратные изображения размером 224 на 224). <br>Так же если ввести при задании объекта значение RGB_presentation = True, то выходной размер изображений станет равен (3, Size, Size) вместо классического для чб снимков тензора размером (1, Size, Size). По умолчанию RGB_presentation = False. Данная функция будет нужна при Transfer Learning, так как предобученные сети имеют первый сверточный слой трехканальным, так что для использования готовых сетей можно просто сделать дублирование одного канала еще 2 раза чтобы изображение черно-белое можно было считать цветным. При выборе RGB_presentation = True тензоры, хранящиеся в атрибутах mean_nums и std_nums тоже меняют размерность


__Пример работы с preprocessing_func:__

In [9]:
data = datasets_bilder.preprocessing_func(size = 200, RGB_presentation = True)

In [10]:
dataset_test = data.test
dataset_train = data.train
dataset_test = data.val
mean_nums = data.mean_nums
std_nums = data.std_nums

In [11]:
mean_nums, std_nums

(tensor([0.5455, 0.5455, 0.5455]), tensor([0.2587, 0.2587, 0.2587]))

In [12]:
a, _ = dataset_test[0]
a.shape

torch.Size([3, 200, 200])

Зададим вычислительное устройство:

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
batch_size = 8 # зададим размер батча

Проверим правильность загруженных данных:

In [ ]:
def imshow(img):
    img = img *std_nums + mean_nums  
    img = img.cpu().numpy()
    plt.imshow(np.transpose(img, (1, 2, 0)))  
    plt.show()

In [ ]:
dataloader_test = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
classes = ['norm','pneumonia']
image_batch, label_batch = next(iter(dataloader_test))
image_batch, label_batch = image_batch.to(device), label_batch.to(device)
print('test:')
plt.figure(figsize=(15, 9), dpi=80)
# выводим изображения в батче
imshow(torchvision.utils.make_grid(image_batch))
print('GroundTruth: ', ',  '.join( classes[int(label_batch[j])] for j in range(batch_size)))

In [ ]:
dataloader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
image_batch, label_batch = next(iter(dataloader_test))
image_batch, label_batch = image_batch.to(device), label_batch.to(device)
print('train:')
plt.figure(figsize=(15, 9), dpi=80)
# выводим изображения в батче
imshow(torchvision.utils.make_grid(image_batch))
print('GroundTruth: ', ',  '.join( classes[int(label_batch[j])] for j in range(batch_size)))

In [ ]:
dataloader_val = DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=True, num_workers=0)

In [18]:
#продолжение следует ...